# Simple Example Script to simulate glitching attacks with angr
This script aims to show the basic usage of angr in combination with some custom scripts to find glitchpoints in binaries. 

In [1]:
# Make sure that custom scripts can be imported
import os
import sys
module_path = os.path.abspath(os.path.join('../py_scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [37]:
# import modules
import angr
import archinfo
import itertools
# import custom scripts
import common
from timeout import TimeLimitedExecution

### Loading the binary
First the binary needs to be loaded. Angr provides a loader that can load elf, mach-o and pe files as well as the blob loader which can load arbitrary binaries.

In [32]:
# Load a elf binary
proj = angr.Project("../example_binaries/out/02_multiglitch")
# When dealing with an non elf/mach-o,pe file (e.g. an arm firmware) additional information needs to be provided:
# proj = angr.Project("./path_to_custom/binary", main_opts={'backend': 'blob', 'arch': 'arm', 'base_addr': 0x40000, 'entry_point': 0x40000}"
common.project_info(proj)

WARNING | 2022-01-20 10:22:42,168 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


############ project info ############
file: ../example_binaries/out/02_multiglitch
arch: <Arch AARCH64 (LE)>
entry: 0x400660
min_addr: 0x400000
max_addr: 0x807fff
############ project info ############


## State preparation
Next the starting point (state) needs to be configured. It can be either an entry state starting at the beginning or a blank state starting at a specific adress. Furthermore statemodifications can be applied to for example simulate values being set in memory.

In [33]:
def get_state():
    # add some options for angr
    state_options = {angr.options.SYMBOL_FILL_UNCONSTRAINED_MEMORY, angr.options.SYMBOL_FILL_UNCONSTRAINED_REGISTERS}
    
    # start with an initial state
    state = proj.factory.entry_state(add_options=state_options)
    # Don't start at entry point of the elf binary, start at the beginning of main
    #state = proj.factory.blank_state(addr=0x40076c, add_options=state_options)
    
    #######
    #State modification section see: https://docs.angr.io/core-concepts/states#review-reading-and-writing-memory-and-registers
    #######
    state.memory.store(0x1FE, 0x87654321, 4, endness=archinfo.Endness.LE)
    state.memory.store(0x2FE, 0x87654321, 4, endness=archinfo.Endness.LE)
    return state

## Find conditions
Next the find condition needs to be configured. It can be either an address that angr should be searching for or a more complex scenarion with a custom find function (shown in notebook TODO). Furthermore it is possible to add multiple avoid addresses witch should be avoided when angr tries to find a path. In the basic example a path to ```0x4007e4``` should be found since it is only reached when CRP is successfully disabled.

In [36]:
# simple find
find=0x4007e0

## First attempt to find a path without a glitch

The following snippet tries to find a path considering the find conditions. Normaly this yields no results as we can't reach our conditions without applying a glitch.

In [35]:
# Get the state from the state functions which was definied previously
state = get_state()
# get a simulation manager
simgr = proj.factory.simgr(state)
# explore, starting from the current state and try to find the find address while avoiding the avoid addresses
simgr.explore(find=find, num_find=5)
simgr.found

[]

## Glitching Addresses
Rather than trying to glitch every instruction in the binary we only try to insert glitches at specific, user defined addresses to keep the searchspace as small as possible. In the basic example only the instructions inside the main function are relevant so these are defined in the glitch_addresses list.

In [41]:
# Add all instructions from 0x40076c to 0x4007f0
glitch_addresses=[addr for addr in range(0x40076c, 0x4007f4, 4)]
glitch_addresses_2x=[e for e in itertools.product(glitch_addresses, glitch_addresses) if e[0] != e[1]]
# Pretty print them:
[f"{hex(addr[0])}|{hex(addr[1])}" for addr in glitch_addresses_2x]

['0x40076c|0x400770',
 '0x40076c|0x400774',
 '0x40076c|0x400778',
 '0x40076c|0x40077c',
 '0x40076c|0x400780',
 '0x40076c|0x400784',
 '0x40076c|0x400788',
 '0x40076c|0x40078c',
 '0x40076c|0x400790',
 '0x40076c|0x400794',
 '0x40076c|0x400798',
 '0x40076c|0x40079c',
 '0x40076c|0x4007a0',
 '0x40076c|0x4007a4',
 '0x40076c|0x4007a8',
 '0x40076c|0x4007ac',
 '0x40076c|0x4007b0',
 '0x40076c|0x4007b4',
 '0x40076c|0x4007b8',
 '0x40076c|0x4007bc',
 '0x40076c|0x4007c0',
 '0x40076c|0x4007c4',
 '0x40076c|0x4007c8',
 '0x40076c|0x4007cc',
 '0x40076c|0x4007d0',
 '0x40076c|0x4007d4',
 '0x40076c|0x4007d8',
 '0x40076c|0x4007dc',
 '0x40076c|0x4007e0',
 '0x40076c|0x4007e4',
 '0x40076c|0x4007e8',
 '0x40076c|0x4007ec',
 '0x40076c|0x4007f0',
 '0x400770|0x40076c',
 '0x400770|0x400774',
 '0x400770|0x400778',
 '0x400770|0x40077c',
 '0x400770|0x400780',
 '0x400770|0x400784',
 '0x400770|0x400788',
 '0x400770|0x40078c',
 '0x400770|0x400790',
 '0x400770|0x400794',
 '0x400770|0x400798',
 '0x400770|0x40079c',
 '0x400770

## Enumerate the glitch_addresses to find a suitable glitchpoint

In [45]:
results = []
for glitch_addr in glitch_addresses_2x:
    print(f"applying glitch to {hex(glitch_addr[0])}|{hex(glitch_addr[1])}")
    
    # get a fresh state + simulation manager
    state = get_state();
    simgr = proj.factory.simgr(state)
    # Set a timelimit to each execution so we don't run into endless loops
    tl = TimeLimitedExecution(time_limit=3000)
    simgr.use_technique(tl)
    
    # actually insert the glitch
    for x in glitch_addr:
        proj.hook(x, common.hook_nop, length=4)
    
    # explore, starting from the current state and try to find the find address while avoiding the avoid addresses
    simgr.explore(find=find, avoid=avoid_addrs, num_find=5)
    if len(simgr.found)>0:
        print(f"\033[92mfound a glitch at: {hex(glitch_addr[0])}|{hex(glitch_addr[1])}\033[0m")
        results.append((glitch_addr, simgr.found))

    for x in glitch_addr:
        proj.unhook(x)

results

applying glitch to 0x40076c|0x400770
applying glitch to 0x40076c|0x400774
applying glitch to 0x40076c|0x400778
applying glitch to 0x40076c|0x40077c
applying glitch to 0x40076c|0x400780


WARNING | 2022-01-20 10:32:11,749 | angr.project | Address is already hooked, during hook(0x400780, <function hook_nop at 0x11becb700>). Not re-hooking.


applying glitch to 0x40076c|0x400784
applying glitch to 0x40076c|0x400788
applying glitch to 0x40076c|0x40078c
applying glitch to 0x40076c|0x400790
applying glitch to 0x40076c|0x400794


WARNING | 2022-01-20 10:32:13,925 | angr.project | Address is already hooked, during hook(0x400798, <function hook_nop at 0x11becb700>). Not re-hooking.


applying glitch to 0x40076c|0x400798
applying glitch to 0x40076c|0x40079c
applying glitch to 0x40076c|0x4007a0
applying glitch to 0x40076c|0x4007a4
applying glitch to 0x40076c|0x4007a8
applying glitch to 0x40076c|0x4007ac
applying glitch to 0x40076c|0x4007b0
applying glitch to 0x40076c|0x4007b4
applying glitch to 0x40076c|0x4007b8
applying glitch to 0x40076c|0x4007bc
applying glitch to 0x40076c|0x4007c0
applying glitch to 0x40076c|0x4007c4
applying glitch to 0x40076c|0x4007c8
applying glitch to 0x40076c|0x4007cc
applying glitch to 0x40076c|0x4007d0
applying glitch to 0x40076c|0x4007d4
applying glitch to 0x40076c|0x4007d8
applying glitch to 0x40076c|0x4007dc
applying glitch to 0x40076c|0x4007e0
applying glitch to 0x40076c|0x4007e4
applying glitch to 0x40076c|0x4007e8
applying glitch to 0x40076c|0x4007ec
applying glitch to 0x40076c|0x4007f0
applying glitch to 0x400770|0x40076c
applying glitch to 0x400770|0x400774
applying glitch to 0x400770|0x400778
applying glitch to 0x400770|0x40077c
a

[((4196224, 4196248), [<SimState @ 0x4007e0>]),
 ((4196224, 4196260), [<SimState @ 0x4007e0>]),
 ((4196224, 4196296), [<SimState @ 0x4007e0>]),
 ((4196224, 4196308), [<SimState @ 0x4007e0>]),
 ((4196224, 4196312), [<SimState @ 0x4007e0>]),
 ((4196228, 4196248), [<SimState @ 0x4007e0>]),
 ((4196228, 4196260), [<SimState @ 0x4007e0>]),
 ((4196228, 4196296), [<SimState @ 0x4007e0>]),
 ((4196228, 4196308), [<SimState @ 0x4007e0>]),
 ((4196228, 4196312), [<SimState @ 0x4007e0>]),
 ((4196240, 4196244), [<SimState @ 0x4007e0>]),
 ((4196240, 4196248), [<SimState @ 0x4007e0>]),
 ((4196240, 4196260), [<SimState @ 0x4007e0>]),
 ((4196240, 4196296), [<SimState @ 0x4007e0>]),
 ((4196240, 4196308), [<SimState @ 0x4007e0>]),
 ((4196240, 4196312), [<SimState @ 0x4007e0>]),
 ((4196244, 4196240), [<SimState @ 0x4007e0>]),
 ((4196244, 4196276), [<SimState @ 0x4007e0>]),
 ((4196244, 4196280), [<SimState @ 0x4007e0>]),
 ((4196248, 4196224), [<SimState @ 0x4007e0>]),
 ((4196248, 4196228), [<SimState @ 0x400